In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
import kagglehub
import os

In [85]:
path = kagglehub.dataset_download("quantbruce/real-estate-price-prediction")

print("Path to dataset files:", path)
print(os.listdir(path))

Path to dataset files: /kaggle/input/real-estate-price-prediction
['Real estate.csv']


In [86]:
df= pd.read_csv('/root/.cache/kagglehub/datasets/quantbruce/real-estate-price-prediction/versions/1/Real estate.csv')

In [87]:
df.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [88]:
df.drop(['No'],inplace=True,axis=1)

In [89]:
df

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1
...,...,...,...,...,...,...,...
409,2013.000,13.7,4082.01500,0,24.94155,121.50381,15.4
410,2012.667,5.6,90.45606,9,24.97433,121.54310,50.0
411,2013.250,18.8,390.96960,7,24.97923,121.53986,40.6
412,2013.000,8.1,104.81010,5,24.96674,121.54067,52.5


In [90]:
df.isnull().sum()

,0
X1 transaction date,0
X2 house age,0
X3 distance to the nearest MRT station,0
X4 number of convenience stores,0
X5 latitude,0
X6 longitude,0
Y house price of unit area,0


In [91]:
df.duplicated().sum()

np.int64(0)

In [92]:
df.dtypes

,0
X1 transaction date,float64
X2 house age,float64
X3 distance to the nearest MRT station,float64
X4 number of convenience stores,int64
X5 latitude,float64
X6 longitude,float64
Y house price of unit area,float64


In [93]:
x = df.drop(['Y house price of unit area'],axis=1)
y = df['Y house price of unit area']

In [94]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [95]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [96]:
predictions = lr.predict(x_test)

In [97]:
print("R2 score of model is ", r2_score(y_test,predictions)) #r2 score is 0.5 which is bad so lets use polynomial

R2 score of model is  0.5155270069624013


In [98]:
poly = PolynomialFeatures(degree=2) #degree is 2 lets change it to 3 and result will get worst so 2 is best as of now
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test) #transforming the values to power to get curve

In [99]:
lr.fit(x_train_poly,y_train)

LinearRegression()

In [100]:
predictionsN = lr.predict(x_test_poly)

In [101]:
print("R2 score is ", r2_score(y_test,predictionsN)) #now the error is so good like amazing lets change the degree

R2 score is  0.5640828016384256


In [102]:
poly = PolynomialFeatures(degree=2,include_bias=False) #degree is 2 lets change it to 3 and result will get worst so 2 is best as of now
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test) #transforming the values to power to get curve
lr.fit(x_train_poly,y_train)
predictionsN = lr.predict(x_test_poly)
print("R2 score is ", r2_score(y_test,predictionsN))

R2 score is  0.5640830345517103


<h2> Score is bad let's apply KNN and see </h2>

In [103]:
from sklearn.neighbors import KNeighborsRegressor

In [104]:
Sc = StandardScaler()
x_trains = Sc.fit_transform(x_train)
x_tests = Sc.transform(x_test)

In [105]:
kn = KNeighborsRegressor(n_neighbors=9, weights='distance', p = 1)
kn.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=9, p=1, weights='distance')

In [106]:
predictionsk = kn.predict(x_test)

In [107]:
print("R2 score now is ", r2_score(y_test,predictionsk)) #sort of same so leaving it here will come back and try to do it better

R2 score now is  0.6104839464319609


In [108]:
from sklearn.linear_model import Ridge

In [109]:
r = Ridge(alpha=0.0001)
r.fit(x_train,y_train)

Ridge(alpha=0.0001)

In [110]:
rpredict = r.predict(x_test)

In [111]:
print("R2 score for predict is ", r2_score(y_test,rpredict))

R2 score for predict is  0.5154820394551752
